In [1]:
import pandas as pd
import joblib

from src import *

In [2]:
## First, read parameter file. This file contains some parameters that can be changed.
paraFile = "paraInit.csv"
paras = pd.read_csv(paraFile, sep=",", header=0)
useDerived = paras.loc[paras["pName"] == "derivedFeatFlag", "pValue"].values[0]
verbose = 0  ## Sets level of verbosity for some functions - 0 means not very

In [3]:
userDataFile = "userData.csv"
userDataMat = pd.read_csv(userDataFile)
userDataMat = digiCot(userDataMat)

> Digitizing cotinine data ... Done



In [4]:
codeBookFile = "codebook_linAge2.csv"
codeBook = pd.read_csv(codeBookFile)

incList = markIncsFromCodeBook(codeBook)

boxCox_lam = pd.read_csv("logNoLog.csv").iloc[1:2, :]


dataMat_trans = pd.read_csv('artifacts/dataMat_trans.csv')
qDataMat = pd.read_csv('artifacts/qDataMat.csv')

vMatDat99_F = pd.read_csv("vMatDat99_F_pre.csv").values
vMatDat99_M = pd.read_csv("vMatDat99_M_pre.csv").values

In [5]:
userDataMat.iloc[0].to_dict()

{'SEQN': 8881.0,
 'RIAGENDR': 1.0,
 'RIDAGEEX': 864.0,
 'BPQ020': 1.0,
 'DIQ010': 1.0,
 'HUQ010': 3.0,
 'HUQ020': 1.0,
 'HUQ050': 3.0,
 'HUQ070': 2.0,
 'KIQ020': 2.0,
 'MCQ010': 2.0,
 'MCQ053': 2.0,
 'MCQ160A': 2.0,
 'MCQ160B': 1.0,
 'MCQ160C': 2.0,
 'MCQ160D': 1.0,
 'MCQ160E': 1.0,
 'MCQ160F': 2.0,
 'MCQ160G': 2.0,
 'MCQ160I': 2.0,
 'MCQ160J': 1.0,
 'MCQ160K': 2.0,
 'MCQ160L': 2.0,
 'MCQ220': 2.0,
 'OSQ010A': 2.0,
 'OSQ010B': 2.0,
 'OSQ010C': 2.0,
 'OSQ060': 2.0,
 'PFQ056': 2.0,
 'BPXPLS': 68.0,
 'BPXSAR': 111.0,
 'BPXDAR': 50.0,
 'BMXBMI': 31.31,
 'URXUMASI': 96.9,
 'URXUCRSI': 18210.0,
 'LBDIRNSI': 19.87,
 'LBDTIBSI': 54.95,
 'LBXPCT': 36.2,
 'LBDFERSI': 81.0,
 'LBDFOLSI': 40.5,
 'LBDB12SI': 418.45,
 'LBXCOT': 2.0,
 'LBDTCSI': 3.83,
 'LBDHDLSI': 0.95,
 'LBXWBCSI': 6.6,
 'LBXLYPCT': 26.5,
 'LBXMOPCT': 8.5,
 'LBXNEPCT': 60.1,
 'LBXEOPCT': 4.3,
 'LBXBAPCT': 0.6,
 'LBDLYMNO': 1.7,
 'LBDMONO': 0.6,
 'LBDNENO': 4.0,
 'LBDEONO': 0.3,
 'LBDBANO': 0.0,
 'LBXRBCSI': 4.99,
 'LBXHGB': 15.1,
 'L

In [6]:
dataFileName = "mergedDataNHANES9902.csv"
masterData = pd.read_csv(dataFileName)
dataMat = dropCols(masterData, incList) 
qDataMat = qDataMatGen(masterData, incList)

coxCovsTrainM = pd.read_csv('artifacts/coxCovsTrainM.csv')
coxCovsTrainF = pd.read_csv('artifacts/coxCovsTrainF.csv')

In [7]:
# Load
coxModelF = joblib.load("artifacts/cox_full_F.joblib")
nullModelF = joblib.load("artifacts/cox_null_F.joblib")

coxModelM = joblib.load("artifacts/cox_full_M.joblib")
nullModelM = joblib.load("artifacts/cox_null_M.joblib")

In [8]:
dataMat_user = dropCols(userDataMat, incList)  ## User data matrix for clock


qDataMat_user = qDataMatGen(userDataMat, incList)

sex_user = qDataMat_user["RIAGENDR"].values
initAge_user = qDataMat_user["RIDAGEEX"].values

In [9]:
qDataMat_user.iloc[0].to_dict()

{'SEQN': 8881,
 'RIAGENDR': 1,
 'RIDAGEEX': 864,
 'BPQ020': 1,
 'DIQ010': 1,
 'HUQ010': 3,
 'HUQ020': 1,
 'HUQ050': 3,
 'HUQ070': 2,
 'KIQ020': 2,
 'MCQ010': 2,
 'MCQ053': 2,
 'MCQ160A': 2,
 'MCQ160B': 1,
 'MCQ160C': 2,
 'MCQ160D': 1,
 'MCQ160E': 1,
 'MCQ160F': 2,
 'MCQ160G': 2,
 'MCQ160I': 2,
 'MCQ160J': 1,
 'MCQ160K': 2,
 'MCQ160L': 2,
 'MCQ220': 2,
 'OSQ010A': 2,
 'OSQ010B': 2,
 'OSQ010C': 2,
 'OSQ060': 2,
 'PFQ056': 2}

In [10]:
if useDerived:
    print("> Populating derived features ... ", end="")
    print(" fs scores ...", end="")
    
    ######### FS scores
    ## NHANES DATA
    fs1Score = popPCFIfs1(qDataMat)
    fs2Score = popPCFIfs2(qDataMat)
    fs3Score = popPCFIfs3(qDataMat)
    dataMat['fs1Score'] = fs1Score
    dataMat['fs2Score'] = fs2Score
    dataMat['fs3Score'] = fs3Score
    
    ## USER DATA
    fs1Score = popPCFIfs1(qDataMat_user)
    fs2Score = popPCFIfs2(qDataMat_user)
    fs3Score = popPCFIfs3(qDataMat_user)
    dataMat_user['fs1Score'] = fs1Score
    dataMat_user['fs2Score'] = fs2Score
    dataMat_user['fs3Score'] = fs3Score
    
    ######### LDL scores
    ## LDL values
    print(" LDLV ...", end="")
    LDLV = populateLDL(dataMat, qDataMat)
    dataMat['LDLV'] = LDLV
    
    ## USER DATA
    LDLV = populateLDL(dataMat_user, qDataMat_user)
    dataMat_user['LDLV'] = LDLV
    
    ######### Urine albumin to creatinine ratio
    ## Urine Albumin Creatinine ratio
    print(" Albumin Creatinine ratio ... ", end="")
    creaVals = dataMat["URXUCRSI"].values
    albuVals = dataMat["URXUMASI"].values
    crAlbRat = albuVals / (creaVals * 1.1312 * 10**-4)
    dataMat['crAlbRat'] = crAlbRat
    
    ## USER DATA
    creaVals = dataMat_user["URXUCRSI"].values
    albuVals = dataMat_user["URXUMASI"].values
    crAlbRat = albuVals / (creaVals * 1.1312 * 10**-4)
    dataMat_user['crAlbRat'] = crAlbRat

> Populating derived features ...  fs scores ... LDLV ... Albumin Creatinine ratio ... 

In [11]:
dataMat_trans_user = boxCoxTransform(boxCox_lam, dataMat_user)

> Applying boxCox transformed  ... Done


In [12]:
dataMat_trans_user_R =  pd.read_csv("dataMat_trans_userR.csv")

In [13]:
pd.testing.assert_frame_equal(dataMat_trans_user.drop(['LBDTCSI', 'LBDHDLSI', 'LBDSTRSI'], axis=1), dataMat_trans_user_R[:2], check_dtype=False)

In [14]:
qDataMat_R = pd.read_csv('qDataMat_R.csv')
qDataMat_user_R = pd.read_csv('qDataMat_user_R.csv')

In [15]:
pd.testing.assert_frame_equal(qDataMat_user_R[:2], qDataMat_user, check_dtype=False)

In [16]:
#pd.testing.assert_frame_equal(qDataMat_R, qDataMat, check_dtype=False) #that one is a troublemaker

In [17]:
dataMatNorm_user = normAsZscores_99_young_mf(dataMat_trans_user.drop(['LBDTCSI', 'LBDHDLSI', 'LBDSTRSI'], axis=1), qDataMat_user, dataMat_trans, qDataMat_R)

dataMatNorm_user_R = pd.read_csv('dataMatNorm_user_R.csv')[:2]
#dataMatNorm_user = pd.read_csv('dataMatNorm_user_R.csv')[:2]

In [18]:
pd.testing.assert_frame_equal(dataMatNorm_user, dataMatNorm_user_R[:2], check_dtype=False)

In [19]:
dataMat_user_R.columns[1:]

NameError: name 'dataMat_user_R' is not defined

In [20]:
zScoreMax = 6

dataMatUser_folded = foldOutliers(dataMatNorm_user, zScoreMax)

inputMat_user = dataMatUser_folded.iloc[:, 1:].values


sexSel_user = qDataMat_user["RIAGENDR"].values

inputMat_user_M = inputMat_user[sexSel_user == 1, :]
inputMat_user_F = inputMat_user[sexSel_user == 2, :]




pcMat_user_M = projectToSVD(inputMat_user_M, vMatDat99_M)
pcMat_user_F = projectToSVD(inputMat_user_F, vMatDat99_F)


rowsAll_user = pcMat_user_M.shape[0] + pcMat_user_F.shape[0]
colsAll = nSVs99_M = 59

pcMat_user = np.zeros((rowsAll_user, colsAll))



pcMat_user[sexSel_user == 1, :] = pcMat_user_M
pcMat_user[sexSel_user == 2, :] = pcMat_user_F
pcMat_user = pd.DataFrame(pcMat_user, columns=[f"PC{i+1}" for i in range(nSVs99_M)])

coxCovs_user = np.column_stack([initAge_user, pcMat_user.values, sex_user])
coxCovs_user = pd.DataFrame(coxCovs_user, columns=['chronAge'] + list(pcMat_user.columns) + ['sex_user'])

## Split back into male / female to apply separate models
coxCovs_user_M = coxCovs_user[sex_user == 1]
coxCovs_user_F = coxCovs_user[sex_user == 2]

coxCovs_user_F = pd.DataFrame(coxCovs_user_F)

coxCovs_user_M_R = pd.read_csv('coxCovs_user_M_R.csv')
coxCovs_user_F_R = pd.read_csv('coxCovs_user_F_R.csv')

delBioAge_user_F = calcBioAge_R_equiv(coxModelF, nullModelF, coxCovs_user_F,  coxCovsTrainF)                    
bioAge_user_F = coxCovs_user_F["chronAge"].values + delBioAge_user_F

delBioAge_user_M = calcBioAge_R_equiv(coxModelM, nullModelM, coxCovs_user_M,  coxCovsTrainM)                    
bioAge_user_M = coxCovs_user_M["chronAge"].values + delBioAge_user_M

> Folding in outliers at maximum total zScore: 6 ... Done
> Projecting data into PC coordinates  ... Done
> Projecting data into PC coordinates  ... Done


In [21]:
delBioAge_user_M_R = pd.read_csv("delBioAge_user_M_R.csv")

In [22]:
delBioAge_user_M

array([200.41124048, -95.62768582])

In [23]:
delBioAge_user_M_R.values.flatten()[:2]

array([200.33384823, -95.70520872])

In [24]:
pc_indices = [int(x[2:])-1 for x in coxModelM.feature_names_in_ if 'PC' in x]

beta_full = np.zeros(59)
beta_full[pc_indices] = coxModelM.coef_[1:]
beta_age_null = nullModelM.coef_[0]

beta_age_full = coxModelM.coef_[0]

w_feature_years = (vMatDat99_M @ beta_full)/beta_age_null

w_age = (beta_age_full / beta_age_null) - 1.0



mu_PC = np.zeros(59)
mu_PC[pc_indices] = coxCovsTrainM.mean().loc[coxModelM.feature_names_in_].iloc[1:].values
mu_age = coxCovsTrainM['chronAge'].mean()

mu_Z = mu_PC@vMatDat99_M.T

Z_centered = inputMat_user_M - mu_Z      # shape (n_samples, n_features)
term_features = (Z_centered @ w_feature_years)
term_age = (initAge_user - mu_age) * w_age

alt_delta = term_features + term_age

In [ ]:
alt_delta/12

In [57]:
len(w_feature_years)

59

In [ ]:
nhanes_desc = {
  # Vitals / anthropometry
  "BPXPLS": "Pulse rate (beats/min, 60-sec pulse).",   # CDC BPX doc
  "BPXSAR": "Systolic blood pressure — average reported to examinee (mmHg).",
  "BPXDAR": "Diastolic blood pressure — average reported to examinee (mmHg).",
  "BMXBMI": "Body mass index (kg/m²).",

  # Kidney (urine)
  "URXUMASI": "Urine albumin (microalbumin), SI units (e.g., mg/L→mg/L; often used for UACR).",
  "URXUCRSI": "Urine creatinine, SI units (mmol/L).",

  # Iron panel
  "LBDIRNSI": "Serum iron (µmol/L).",
  "LBDTIBSI": "Total iron binding capacity, TIBC (µmol/L).",
  "LBXPCT":   "Transferrin saturation (%) = (serum iron / TIBC)×100.",
  "LBDFERSI": "Ferritin (µg/L).",

  # Folate / B12 / cotinine
  "LBDFOLSI": "Serum folate (nmol/L).",
  "LBDB12SI": "Vitamin B12 (pmol/L).",
  "LBXCOT":   "Cotinine (ng/mL)—tobacco exposure marker.",

  # CBC (white cells)
  "LBXWBCSI": "White blood cell count (×10⁹/L).",
  "LBXLYPCT": "Lymphocytes (%).",
  "LBXMOPCT": "Monocytes (%).",
  "LBXNEPCT": "Neutrophils (%).",
  "LBXEOPCT": "Eosinophils (%).",
  "LBXBAPCT": "Basophils (%).",
  "LBDLYMNO": "Lymphocytes (×10⁹/L).",
  "LBDMONO":  "Monocytes (×10⁹/L).",
  "LBDNENO":  "Neutrophils (×10⁹/L).",
  "LBDEONO":  "Eosinophils (×10⁹/L).",
  "LBDBANO":  "Basophils (×10⁹/L).",

  # CBC (red cells / platelets)
  "LBXRBCSI": "Red blood cell count (×10¹²/L).",
  "LBXHGB":   "Hemoglobin (g/dL).",
  "LBXHCT":   "Hematocrit (%).",
  "LBXMCVSI": "Mean corpuscular volume, MCV (fL).",
  "LBXMCHSI": "Mean corpuscular hemoglobin, MCH (pg).",
  "LBXMC":    "Mean corpuscular hemoglobin concentration, MCHC (g/dL).",
  "LBXRDW":   "Red cell distribution width (%).",
  "LBXPLTSI": "Platelet count (×10⁹/L).",
  "LBXMPSI":  "Mean platelet volume, MPV (fL).",

  # Inflammation / glycemia / cardiac
  "LBXCRP": "C-reactive protein (mg/L).",
  "LBXGH":  "Glycohemoglobin (HbA1c, %).",
  "SSBNP":  "N-terminal pro-B-type natriuretic peptide (NT-proBNP, pg/mL).",

  # Basic chem (SI set)
  "LBDSALSI": "Albumin (g/L).",
  "LBXSATSI": "Alanine aminotransferase, ALT (U/L).",
  "LBXSASSI": "Aspartate aminotransferase, AST (U/L).",
  "LBXSAPSI": "Alkaline phosphatase (U/L).",
  "LBDSBUSI": "Urea nitrogen (BUN), SI (mmol/L).",
  "LBDSCASI": "Calcium, SI (mmol/L).",
  "LBXSC3SI": "Bicarbonate (total CO₂), SI (mmol/L).",
  "LBDSGLSI": "Glucose, SI (mmol/L).",
  "LBXSLDSI": "Lactate dehydrogenase, LDH (U/L).",
  "LBDSPHSI": "Phosphorus (mmol/L).",
  "LBDSTBSI": "Total bilirubin (µmol/L).",
  "LBDSTPSI": "Total protein (g/L).",
  "LBDSUASI": "Uric acid (µmol/L).",
  "LBDSCRSI": "Creatinine (µmol/L).",
  "LBXSNASI": "Sodium (mmol/L).",
  "LBXSKSI": "Potassium (mmol/L).",
  "LBXSCLSI": "Chloride (mmol/L).",
  "LBDSGBSI": "Globulin (g/L).",

  # Derived / study-specific
  "fs1Score": "Comorbidity/Frailty index.",
  "fs2Score": "Self-rated health × trajectory.",
  "fs3Score": "Healthcare use (past year).",
  "LDLV":     "Calculated LDL cholesterol (Friedewald or NHANES calc; mg/dL).",
  "crAlbRat": "Urine albumin-to-creatinine ratio (UACR).",
}


In [26]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

feature_names = ['BPXPLS', 'BPXSAR', 'BPXDAR', 'BMXBMI', 'URXUMASI', 'URXUCRSI',
       'LBDIRNSI', 'LBDTIBSI', 'LBXPCT', 'LBDFERSI', 'LBDFOLSI', 'LBDB12SI',
       'LBXCOT', 'LBXWBCSI', 'LBXLYPCT', 'LBXMOPCT', 'LBXNEPCT', 'LBXEOPCT',
       'LBXBAPCT', 'LBDLYMNO', 'LBDMONO', 'LBDNENO', 'LBDEONO', 'LBDBANO',
       'LBXRBCSI', 'LBXHGB', 'LBXHCT', 'LBXMCVSI', 'LBXMCHSI', 'LBXMC',
       'LBXRDW', 'LBXPLTSI', 'LBXMPSI', 'LBXCRP', 'LBXGH', 'SSBNP', 'LBDSALSI',
       'LBXSATSI', 'LBXSASSI', 'LBXSAPSI', 'LBDSBUSI', 'LBDSCASI', 'LBXSC3SI',
       'LBDSGLSI', 'LBXSLDSI', 'LBDSPHSI', 'LBDSTBSI', 'LBDSTPSI', 'LBDSUASI',
       'LBDSCRSI', 'LBXSNASI', 'LBXSKSI', 'LBXSCLSI', 'LBDSGBSI', 'fs1Score',
       'fs2Score', 'fs3Score', 'LDLV', 'crAlbRat']

In [28]:
def plot_feature_contribs_interactive_np(
    Z_centered: np.ndarray,
    w_feature_years: np.ndarray,
    raw_features: pd.DataFrame,
    feature_names: list[str],
    subject_idx: int = 0,
    title: str | None = None,
    term_age: float | None = None,
    descriptions: dict | pd.Series | None = None,
):
    """
    Z_centered : np.ndarray (n_samples × n_features)
    w_feature_years : np.ndarray (n_features,)
    feature_names : list of feature codes (len = n_features)
    descriptions : dict or pd.Series mapping feature -> human-readable text
    """
    dfZ = pd.DataFrame(Z_centered, columns=feature_names)
    w = pd.Series(np.asarray(w_feature_years), index=feature_names, name="w_years_per_SD")
    row = dfZ.iloc[subject_idx]

    # construct plotting dataframe
    plot_df = pd.DataFrame({
        "feature": feature_names,
        "description": [descriptions.get(f, f) for f in feature_names] if descriptions is not None else feature_names,
        "z_centered": row.values,
        "w_years_per_SD": w.values,
        "contribution_years": (row.values * w.values)/12,
        "lab_values": raw_features[feature_names].iloc[subject_idx].values
    }).sort_values("contribution_years")

    color_continuous_scale=[(0, "blue"), (0.5, "white"), (1, "red")],
    range_color=[plot_df["contribution_years"].min(),
                 plot_df["contribution_years"].max()]

    vals = plot_df["contribution_years"]
    m = float(np.nanquantile(np.abs(vals), 0.9))
    
    # color scale: red→blue (aging/de-aging)
    fig = px.bar(
        plot_df,
        x="contribution_years",
        y="feature",
        orientation="h",
        title=title or f"Feature contributions for subject #{subject_idx}",
        
        color="contribution_years",
        color_continuous_scale="RdBu_r",         # blue for negative, red for positive
        range_color=[-m, m],                     # <-- symmetric
        color_continuous_midpoint=0,
        hover_data=["description", "z_centered", "w_years_per_SD", "contribution_years"],
         width=1600, height=800
    )

    # nice hover tooltip template
    fig.update_traces(
        customdata=plot_df[["description","lab_values","z_centered","w_years_per_SD"]].values,
        hovertemplate=(
            "<b>%{y}</b><br>%{customdata[0]}<br>"
            "user input: %{customdata[1]}<br>"
            "z (centered): %{customdata[2]:.3f}<br>"
            "weight (yrs/SD): %{customdata[3]:.3f}<br>"
            "contrib (yrs): %{x:.3f}<extra></extra>"
        )
    )

    if term_age is not None:
        fig.add_vline(x=float(term_age)/12, line_dash="dash", line_color="black", annotation_text="Age term")

    fig.update_layout(yaxis=dict(dtick=1))
    return fig


In [29]:
# Assume you already have:
#   Z_centered, w_feature_years, feature_names, term_age
#   and nhanes_desc = {...}  # from the dict I gave you

subject_idx=0

fig = plot_feature_contribs_interactive_np(
    Z_centered,
    w_feature_years,
    dataMat_user,
    feature_names,
    subject_idx=subject_idx,
    title="LinAge2 (M) — feature contributions",
    term_age=float(term_age[subject_idx]) if hasattr(term_age, "__getitem__") else term_age,
    descriptions=nhanes_desc
)
fig.show()

NameError: name 'nhanes_desc' is not defined

In [31]:
fig.write_html('figure_example.html', include_plotlyjs="cdn", full_html=True)

NameError: name 'fig' is not defined

In [32]:
from slider_norm import compute_young_median_mad, slider_bounds_from_young_stats

ModuleNotFoundError: No module named 'slider_norm'

In [33]:
coxCovs_user_M_app = pd.read_csv('coxCovs_user_M_app.csv')

In [34]:
coxCovs_user_M

,chronAge,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,sex_user
0,864.0,8.948283,-1.314347,-0.247391,-1.456070,0.661141,1.098362,-0.568712,-1.560459,1.322840,...,-0.055285,0.013903,-0.007151,0.023820,-0.022556,-0.078123,0.016839,0.007282,0.023680,1.0
1,868.0,2.643548,0.963427,3.214944,-0.399939,0.785004,-2.544099,-1.049894,0.496311,-1.293523,...,-0.069387,0.032526,-0.019005,-0.022903,-0.031473,0.018438,-0.001631,-0.013183,-0.008595,1.0


In [35]:
coxCovs_user_M_app

,Unnamed: 0,chronAge,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,sex_user
0,0,33.0,7.784435,-1.848079,-1.084009,-1.949078,0.340711,0.694548,0.271115,-1.729271,...,-0.065633,0.023329,0.017456,0.009401,0.131281,-0.038488,0.030364,-0.01038,0.013265,1.0


In [36]:
dataMat_user_app = pd.read_csv('dataMat_user_app.csv')

In [37]:
a = dataMat_user.iloc[0].to_dict()

In [38]:
b = dataMat_user_app.iloc[0].to_dict()

In [39]:
{x:(a[x], b[x]) for x in a if a[x]!=b[x]}

{'SEQN': (8881.0, 1.0),
 'fs1Score': (0.22727272727272727, 0.0),
 'fs3Score': (3.0, 0.0)}

In [40]:
inputMat_user.shape

(2, 59)

In [41]:
pcMat_user_app = pd.read_csv('pcMat_user_app.csv')

In [42]:
pcMat_user_app

,Unnamed: 0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59
0,0,7.784435,-1.848079,-1.084009,-1.949078,0.340711,0.694548,0.271115,-1.729271,1.161202,...,0.31146,-0.065633,0.023329,0.017456,0.009401,0.131281,-0.038488,0.030364,-0.01038,0.013265


In [43]:
pcMat_user

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59
0,8.948283,-1.314347,-0.247391,-1.456070,0.661141,1.098362,-0.568712,-1.560459,1.322840,-0.873977,...,0.368845,-0.055285,0.013903,-0.007151,0.023820,-0.022556,-0.078123,0.016839,0.007282,0.023680
1,2.643548,0.963427,3.214944,-0.399939,0.785004,-2.544099,-1.049894,0.496311,-1.293523,-1.818883,...,0.089970,-0.069387,0.032526,-0.019005,-0.022903,-0.031473,0.018438,-0.001631,-0.013183,-0.008595


In [44]:
dataMatNorm_user_app = pd.read_csv('dataMatNorm_user_app.csv')

In [45]:
dataMatNorm_user_app

,Unnamed: 0,SEQN,BPXPLS,BPXSAR,BPXDAR,BMXBMI,URXUMASI,URXUCRSI,LBDIRNSI,LBDTIBSI,...,LBDSCRSI,LBXSNASI,LBXSKSI,LBXSCLSI,LBDSGBSI,fs1Score,fs2Score,fs3Score,LDLV,crAlbRat
0,0,1,-0.168622,-0.824376,-2.697959,0.74775,15.478675,0.76646,0.468214,-0.817323,...,0.682154,1.222513,0.607041,-0.168622,1.34898,0.0,0.0,0,-1.10252,3.845386


In [46]:
dataMatNorm_user

,SEQN,BPXPLS,BPXSAR,BPXDAR,BMXBMI,URXUMASI,URXUCRSI,LBDIRNSI,LBDTIBSI,LBXPCT,...,LBDSCRSI,LBXSNASI,LBXSKSI,LBXSCLSI,LBDSGBSI,fs1Score,fs2Score,fs3Score,LDLV,crAlbRat
0,8881,-0.168622,-0.824376,-2.697959,0.747750,6.000000,0.766460,0.468214,-0.817323,0.848222,...,0.682154,1.222513,0.607041,-0.168622,1.34898,0.227273,0.0,3,-1.102520,3.845386
1,9106,-1.686224,2.398186,-0.385423,-1.022086,0.259419,0.180447,1.903960,-0.213258,1.941713,...,0.000000,-0.042156,-0.303520,-0.438418,-0.89932,0.090909,0.0,2,-0.131673,0.279405


In [47]:
a = dataMat_trans_user.iloc[0].to_dict()

In [48]:
dataMat_trans_user_app = pd.read_csv('dataMat_trans_user_app.csv')

In [49]:
b = dataMat_trans_user_app.iloc[0].to_dict()

In [50]:
{x:(a[x], b[x]) for x in a if a[x]!=b[x]}

{'SEQN': (8881.0, 1.0),
 'URXUCRSI': (9.809726172721653, 9.809726172721652),
 'LBDFOLSI': (3.7013019741124933, 3.701301974112493),
 'LBDHDLSI': (-0.05129329438755058, -0.0512932943875505),
 'LBXWBCSI': (1.8870696490323797, 1.88706964903238),
 'LBXCRP': (-0.05129329438755058, -0.0512932943875505),
 'LBDSTRSI': (0.026641930946421092, 0.026641930946421),
 'fs1Score': (0.22727272727272727, 0.0),
 'fs3Score': (3.0, 0.0)}

In [51]:
alt_delta/12

array([16.70093671, -7.96897382])

In [52]:
userDataMat

,SEQN,RIAGENDR,RIDAGEEX,BPQ020,DIQ010,HUQ010,HUQ020,HUQ050,HUQ070,KIQ020,...,LBDSPHSI,LBDSTBSI,LBDSTPSI,LBDSTRSI,LBDSUASI,LBDSCRSI,LBXSNASI,LBXSKSI,LBXSCLSI,LBDSGBSI
0,8881,1,864,1,1,3,1,3,2,2,...,1.518,13.7,79,1.027,303.3,79.6,142.4,4.35,101.6,36
1,9106,1,868,2,2,2,3,2,2,2,...,1.227,10.3,72,0.960,291.5,70.7,139.4,4.08,100.8,26


In [ ]:
pd.Series(w_feature_years, index=feature_names)

In [64]:
sum([True, True, False])

2

In [65]:
dataMat

,SEQN,BPXPLS,BPXSAR,BPXDAR,BMXBMI,URXUMASI,URXUCRSI,LBDIRNSI,LBDTIBSI,LBXPCT,...,LBDSCRSI,LBXSNASI,LBXSKSI,LBXSCLSI,LBDSGBSI,fs1Score,fs2Score,fs3Score,LDLV,crAlbRat
0,1,NaN,NaN,NaN,14.90,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.045455,0.0,3.0,0.0000,NaN
1,2,68.0,98.0,56.0,24.90,9.1,12818.0,11.64,71.60,16.3,...,61.90,144.1,4.06,104.5,27.0,0.136364,0.0,3.0,3.9104,6.275983
2,3,104.0,108.0,63.0,17.63,10.4,11138.0,30.79,81.62,37.7,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,1.0,0.0000,8.254423
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,2.0,0.0000,NaN
4,5,66.0,122.0,83.0,29.10,6.1,15205.0,25.24,60.86,41.5,...,70.70,137.5,4.63,98.9,28.0,0.136364,0.0,3.0,5.3600,3.546533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20999,21000,78.0,94.0,48.0,27.23,2.9,6630.0,15.57,80.55,19.3,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,2.0,0.0000,3.866741
21000,21001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,3.0,0.0000,NaN
21001,21002,80.0,113.0,48.0,20.84,9.8,17326.0,14.14,77.87,18.2,...,NaN,NaN,NaN,NaN,NaN,0.045455,0.0,5.0,0.0000,5.000211
21002,21003,84.0,100.0,46.0,14.98,8.9,14321.0,9.13,56.74,16.1,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,3.0,0.0000,5.493856


In [66]:
qDataMat_user

,SEQN,RIAGENDR,RIDAGEEX,BPQ020,DIQ010,HUQ010,HUQ020,HUQ050,HUQ070,KIQ020,...,MCQ160I,MCQ160J,MCQ160K,MCQ160L,MCQ220,OSQ010A,OSQ010B,OSQ010C,OSQ060,PFQ056
0,8881,1,864,1,1,3,1,3,2,2,...,2,1,2,2,2,2,2,2,2,2
1,9106,1,868,2,2,2,3,2,2,2,...,2,2,2,2,2,2,1,2,2,2
